# 7.2. Случайный лес

Ваша задача - написать класс `random_forest` для решения задачи классификации на основе датасета Ирисов Фишера (`sklearn.datasets.load_iris`), принимающий на вход конструктора аргументы `n_estimators`, `max_depth`, `subspaces_dim` и `random_state`. описание этих аргументов приведено ниже. У этого класса должны быть определены методы `.fit()` и `.predict()`, а также поле `._estimators`, в котором должен храниться список алгоритмов, используемых в ансамбле.

- n_estimators - число деревьев в ансамбле
- max_depth - максимальная глубина каждого дерева в ансамбле
- subspaces_dim - размерность случайного подпространства для каждого дерева

Для создания обучающей подвыборки для каждого из базовых классификаторов, Вы можете воспользоваться классом `sample`, который Вы реализовали в прошлом задании. В случае его использования, не забудьте включить его описание в файл с Вашим решением текущего задания. Мы также предлагаем вам организовать выбор подпространств для каждого дерева посредством заполнения списка `subspace_idx`, в котором будут логироваться выбранные для каждого базового классификатора подпространства.

Замечание: в рамках выполнения данного задания запрещено использовать класс `sklearn.ensemble.RandomForestClassifier`. Такой код не пройдёт проверку.

Подберите также гиперпараметры, на которых ваш алгоритм получает наилучшее качество (с точки зрения метрики accuracy, доли правильных ответов) на тестовой выборке с параметром `test_size`=0.3, задайте их в виде глобальных переменных N_ESTIMATORS, MAX_DEPTH, SUBSPACE_DIM.

Шаблон класса:

In [36]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
np.random.seed(42)

class sample(object):
  def __init__(self, X, n_subspace):
    self.idx_subspace = self.random_subspace(X, n_subspace)

  def __call__(self, X, y):
    idx_obj = self.bootstrap_sample(X)
    X_sampled, y_sampled = self.get_subsample(X, y, self.idx_subspace, idx_obj)
    return X_sampled, y_sampled

  @staticmethod
  def bootstrap_sample(X):
    return np.unique(np.random.choice(X.shape[0], X.shape[0]))

  @staticmethod
  def random_subspace(X, n_subspace):
    return np.sort(np.random.choice(X.shape[1], n_subspace, replace=False))

  @staticmethod
  def get_subsample(X, y, idx_subspace, idx_obj):
    return X[idx_obj][:, idx_subspace], y[idx_obj]



N_ESTIMATORS = 1
MAX_DEPTH = 3
SUBSPACE_DIM = 3

class random_forest(object):
  def __init__(self, n_estimators: int, max_depth: int, subspaces_dim: int, random_state: int):
    self.n_estimators = n_estimators
    self.max_depth = max_depth
    self.subspaces_dim = subspaces_dim
    self.random_state = random_state
    self.clasifiers = [0] * self.n_estimators
    self.idx_subspaces = [0] * self.n_estimators

  def fit(self, X, y):
    for i in range(self.n_estimators):
      s = sample(X, self.subspaces_dim)
      X_new, y_new = s(X, y)
      self.clasifiers[i] = DecisionTreeClassifier(max_depth=self.max_depth, random_state=self.random_state).fit(X_new, y_new)
      self.idx_subspaces[i] = s

  def predict(self, X):
    predictions = [0] * self.n_estimators
    for i in range(self.n_estimators):
      X_new = X[:][:, self.idx_subspaces[i].idx_subspace]
      predictions[i] = self.clasifiers[i].predict(X_new)
    predictions = np.array(predictions)
    num_answers = predictions.shape[1]
    ret = [0] * num_answers
    for i in range(num_answers):
      ret[i] = Counter(predictions[:, i]).most_common(1)[0][0]
    return np.array(ret)

In [51]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import linear_model
from sklearn import metrics
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [52]:
X, y = load_iris(return_X_y=True)
X_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, shuffle=True, random_state=42
)
accuracy = np.empty((20, 20, 4), dtype=np.float64)

for n_estimators in range(1, 21):
    for max_depth in range(1, 21):
        for subspace_dim in range(1, 5):
            clf = random_forest(n_estimators, max_depth, subspace_dim, 42)
            clf.fit(X_train, y_train)
            y_pred = clf.predict(x_test)
            accuracy[
                n_estimators - 1, max_depth - 1, subspace_dim - 1
            ] = accuracy_score(y_test, y_pred)

ind = np.unravel_index(np.argmax(accuracy, axis=None), accuracy.shape)
print(ind)
print(accuracy[ind[0], ind[1], ind[2]])

(0, 2, 2)
1.0


## Примечания

1. В данной задаче запрещено использовать библиотеку pandas.

2. В реализуемых методах запрещается использовать вывод любой информации на экран (в частности, недопустимо использование print()).

# Пример входных и выходных данных

In [37]:
X = np.array([[ 0.47819456, -1.57891216, -0.1018819 ,  1.11113501,  0.20826281,
        -1.11091227,  0.07844205],
       [ 0.11850997,  1.91073022,  0.95574903,  1.35798262,  0.56177995,
         0.26012021,  0.42404407],
       [-0.52304666,  0.75051167, -1.037804  , -0.10105312,  0.08559063,
         0.5102743 , -1.79068927],
       [-0.09078024,  1.62097709,  0.93284371,  1.0386902 , -0.68354252,
        -1.27138661,  0.15060651],
       [ 0.11676701, -0.71769062, -0.80119565,  0.73448495,  1.80728052,
         0.45770337,  0.20689119]])
print(X)

[[ 0.47819456 -1.57891216 -0.1018819   1.11113501  0.20826281 -1.11091227
   0.07844205]
 [ 0.11850997  1.91073022  0.95574903  1.35798262  0.56177995  0.26012021
   0.42404407]
 [-0.52304666  0.75051167 -1.037804   -0.10105312  0.08559063  0.5102743
  -1.79068927]
 [-0.09078024  1.62097709  0.93284371  1.0386902  -0.68354252 -1.27138661
   0.15060651]
 [ 0.11676701 -0.71769062 -0.80119565  0.73448495  1.80728052  0.45770337
   0.20689119]]


In [38]:
X.shape

(5, 7)

In [39]:
y = np.array([1, 0, 1, 1, 0])
print(y, y.shape)

[1 0 1 1 0] (5,)


In [40]:
rf = random_forest(25, 15, 2, 42)
rf.fit(X, y)

In [41]:
preds = rf.predict(X)

In [42]:
preds

array([1, 0, 1, 1, 0])